This notebook demonstrates the local optimization of a filter coupled to a W thermal emitter,
and should reproduce the efficiency reported in Table II "W + F$_{opt}$" in 
"Accelerating the discovery of multi-layer nanostructures with analytic differentiation 
of the transfer matrix equations" by J. F. Varner, D. Wert, A. Matari, R. Nofal, and J. J. Foley IV.

In [11]:
import numpy as np
print(np.exp(-2000))

0.0


In [ ]:
from wptherml.wpml import multilayer
from matplotlib import pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import basinhopping
import time
import numpy as np

### This will define a thick tungsten slab
w_struct = {

        'Material_List': ['Air', 'W', 'Air'],
        'Thickness_List': [0,  900e-9, 0],
        'Lambda_List': [300e-9, 4000e-9, 1000],
        'Temperature': 2700,
        'LIGHTBULB': 1

        }

### create instance of the tungsten emitter structure
w         = multilayer(w_struct)


### Create a global variable called emissivity that will NOT change from the original emissivity of W slab
e_emissivity = w.emissivity_array

### This will define the base filter that will be optimized
structure = {
        'Material_List' : ['Air', 'Ta2O5','SiO2','Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5','SiO2', 'Ta2O5', 'SiO2', 'Air'],
        ### Thicknesses just chosen arbitrarily, replace with "optimal" values
        'Thickness_List': [0, 18e-9, 47e-9, 156e-9, 212e-9, 178e-9, 23e-9, 51e-9, 224e-9, 150e-9, 205e-9, 258e-9, 187e-9, 243e-9, 190e-9, 266e-9, 215e-9, 153e-9, 227e-9, 154e-9, 226e-9, 152e-9, 245e-9, 24e-9, 229e-9, 263e-9, 190e-9, 257e-9, 200e-9, 260e-9, 224e-9, 27e-9,  229e-9, 154e-9, 219e-9, 274e-9, 198e-9, 405e-9, 211e-9, 166e-9, 233e-9, 47e-9, 66e-9, 17e-9, 125e-9, 153e-9, 237e-9, 151e-9, 225e-9, 147e-9, 193e-9, 127e-9, 214e-9, 135e-9, 173e-9, 112e-9, 165e-9, 130e-9, 223e-9, 130e-9, 163e-9, 112e-9, 164e-9, 114e-9, 167e-9, 121e-9, 378e-9, 114e-9, 160e-9, 113e-9, 174e-9, 117e-9, 211e-9, 23e-9, 221e-9, 261e-9, 399e-9, 266e-9, 390e-9, 28e-9, 18e-9, 367e-9, 198e-9, 302e-9, 28e-9, 33e-9, 426e-9, 31e-9, 15e-9, 222e-9, 96e-9, 0 ],
        'Lambda_List': [300e-9, 4000e-9, 1000],
        'Temperature': 2700,
        'LIGHTBULB': 1

        }
### create instance called cc
cc = multilayer(structure)

### get initial luminous efficiency of W-emitter / filter pair
cc.luminous_efficiency_filter(e_emissivity)
print(" Filtered Multilayer Luminous Efficiency", cc.luminous_efficiency_val*100)

### How many elements in the filter will be varied over?
length = len(cc.luminous_efficiency_grad)
print("length is ",length)

### given an array of thicknesses for the filter, update the filter and
### re-compute the luminous efficiency of the W-emitter / filter pair
def update_multilayer(x0):
    ### use gradient_list to define which layers are updated!
    dim = len(x0)
    for i in range(1,dim+1):
        cc.d[i] = x0[i-1]*1e-9
        
    cc.fresnel()
    cc.thermal_emission()
    cc.luminous_efficiency_filter(e_emissivity)
    ### return the negative of the efficiency since the 
    ### scipy minimize functions find the MINIMUM not the MAXIMUM...
    ### we of course want the MAXIMUM efficiency which is the same
    ### as the MINIMUM of the negative of the efficiency
    return -cc.luminous_efficiency_val*100

### given an array of thicknesses for the filter, update the filter
### and compute the gradient of its transmissivity...
### use that to compute the gradient of the luminous efficiency
### return the negative of the luminous efficiency gradient 
### scaled by 10^-7 
def analytic_grad(x0):
    dim = len(x0)
    g = np.zeros(dim)
    cur = update_multilayer(x0)
    cc.fresnel_prime()
    cc.luminous_efficiency_filter_prime(e_emissivity)
    g = cc.luminous_efficiency_grad
    return -g*1e-7

### Function that gets the negative of the efficiency and the 
### negative of the gradient for use in the l-bfgs-b algorithm
### also prints out the time for timing purposes!
def SuperFunc(x0):
    en = update_multilayer(x0)
    c_time = time.time()
    print(en,",",c_time)
    gr = analytic_grad(x0)
    return en, gr


# the bounds for L-BFGS-B updates!
bfgs_xmin = np.ones(length)
bfgs_xmax = 405*np.ones(length)

# rewrite the bounds in the way required by L-BFGS-B
bfgs_bounds = [(low, high) for low, high in zip(bfgs_xmin, bfgs_xmax)]

### initialize the solution vector xs to be the thicknesses from 
### the structure dictionary
xs = np.zeros(length)
for i in range(0,length):
    xs[i] = cc.d[i+1]*1e9

### print out initial solution vector and initial efficiency
print("xs is ")
print(xs)
print("efficiency is ",update_multilayer(xs))

### run l-bfgs-b algorithm!
ret = minimize(SuperFunc, xs, method="L-BFGS-B", jac=True, bounds=bfgs_bounds)

### print optimal solution and its efficiency!
print(ret.x)
print(update_multilayer(ret.x))
                                    